<a id='0'></a>
# Table of Contents

1. [Project Goal & Dataset](#pgd)
2. [PySpark Installation](#pysi)
3. [Data Analysis](#da)

<a id='pgd'></a>
# Project Goal

Showcase how pyspark can be used for answering analytics questions for a dataset from Uber case Study.

Dataset Source: https://www.kaggle.com/datasets/nanasahebshinde/uber-case-study

### Data Dictionary:

| Column  | Definition |
| --- | --- |
| Date |  |
| Time(Local) | Hour of the day |
| App Openings (Eyeballs) | Customers who launch the app looking for riders. It is a good measure of demand |
| Zeroes | Customers who open the app and see no cars in the area. |
| Requests | Customers who make requests for a car. |
| Completed Trip | The point from when a customer is picked |
| Unique Drivers | Drivers who logged in |

### Questions we would answer

1.  Which date had the most completed trips during the two-week period?
2.  What was the highest number of completed trips within a 24-hour period?
3.  Which hour of the day had the most requests during the two-week period?
4.  What percentages of all zeroes during the two-week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)? Tip: The local time value is the start of the hour (e.g. `15` is the hour from 3:00 pm - 4:00 pm)
5.  What is the weighted average ratio of completed trips per driver during the two-week period? Tip: "Weighted average" means your answer should account for the total trip volume in each hour to determine the most accurate number in the whole period.
6.  In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two-week period in terms of unique requests? A new shift starts every 8 hours. Assume that a driver will work the same shift each day.
7.  True or False: Driver supply always increases when demand increases during the two-week period. Tip: Visualize the data to confirm your answer if needed.
8.  In which 72-hour period is the ratio of Zeroes to Eyeballs the highest?
9.  If you could add 5 drivers to any single hour of every day during the two-week period, which hour should you add them to? Hint: Consider both rider eyeballs and driver supply when choosing
10. Looking at the data from all two weeks, which time might make the most sense to consider a true "end day" instead of midnight? (i.e when are supply and demand at both their natural minimums)

[back to top](#0)

<a id='pysi'></a>
# PySpark Installation \[Windows\]

1. Download and Install python. Only python 3.10 seems to be working with latest spark. Download Link: https://www.python.org/downloads/release/python-3100/
2. Dowload and install JDK8 installer from the Oracle website. Download Link: https://www.oracle.com/java/technologies/javase/javase-jdk8-downloads.html
Add **JAVA_HOME** to your environment variables and set it to downloaded extracted jdk.
5. Download latest Apache Spark from:  https://spark.apache.org/downloads.html. Set SPARK_HOME in environment variables where spark is extracted.
6. Setup Hadoop
   1. create a folder with following structure anywhere: **hadoop/bin**
   2. Download and extract only **winutils.exe** from here: https://github.com/steveloughran/winutils  to this **hadoop/bin** folder. Latest version works. Download link: https://github.com/steveloughran/winutils/tree/master/hadoop-3.0.0/bin
   3. set HADOOP_HOME environment variable to this **path_where_u_create this folder/hadoop** directory.
7. Install pyspark: pip install pyspark
8. Test if pyspark works.

   The following code should work

   `>>> from pyspark.sql import SparkSession`
   
    `>>> spark = SparkSession.builder.master("local").appName("PySpark Installation Test").getOrCreate()`

    `>>> df = spark.createDataFrame([(1, "Hello"), (2, "World")], ["id", "message"])`

    `>>> df.show()`


[back to top](#0)

<a id='da'></a>
# Data Analysis


In [1]:
from pyspark.sql.functions import max
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("PySpark Installation Test").getOrCreate()

# Read the data from CSV file
uber = spark.read.csv("dataset.csv", header=True, inferSchema=True)
uber.show(n=5)

+---------+------------+--------+------+---------------+--------+--------------+
|     Date|Time (Local)|Eyeballs|Zeroes|Completed Trips|Requests|Unique Drivers|
+---------+------------+--------+------+---------------+--------+--------------+
|10-Sep-12|           7|       5|     0|              2|       2|             9|
|10-Sep-12|           8|       6|     0|              2|       2|            14|
|10-Sep-12|           9|       8|     3|              0|       0|            14|
|10-Sep-12|          10|       9|     2|              0|       1|            14|
|10-Sep-12|          11|      11|     1|              4|       4|            11|
+---------+------------+--------+------+---------------+--------+--------------+
only showing top 5 rows



In [2]:
# 1. Which date had the most completed trips during the two-week period?
# Group the data by date and sum the completed trips
completed_trips_by_date = uber.groupBy("Date").sum("Completed Trips")

# Find the date with the most completed trips
date_with_most_completed_trips = completed_trips_by_date \
    .orderBy("sum(Completed Trips)", ascending=False) \
    .select("Date") \
    .first()["Date"]

print(date_with_most_completed_trips)

22-Sep-12


In [3]:
# 2. What was the highest number of completed trips within a 24-hour period?
# Question is not clear one: it could be the max no of trips evry day, which accounts for 24 hours or it could
# be a window of 24 hour starting from current hour. I will use the second one.
# Since there are no missing hours on any day, the simple window selecting 24 rows would be fine
from pyspark.sql import Window
from pyspark.sql.functions import sum
w = Window.rowsBetween(0, 23)
uber.withColumn('ra', sum('Completed Trips').over(w)).orderBy("ra", ascending=False).first()['ra']

278

In [4]:
#3 .Which hour of the day had the most requests during the two-week period?
from pyspark.sql.functions import sum
uber.groupBy("Time (Local)").sum("Requests").orderBy("sum(Requests)", ascending=False).first()[0]

23

In [5]:
#4. What percentages of all zeroes during the two-week period occurred on weekend 
# (Friday at 5 pm to Sunday at 3 am)? Tip: The local time value is the start of the 
# hour (e.g. 15 is the hour from 3:00 pm - 4:00 pm)
# dayofweek is 1 for Sunday
from pyspark.sql.functions import dayofweek,to_timestamp

uwdw = uber.withColumn('dw',dayofweek(to_timestamp(uber['Date'],"dd-MMM-yy")))
weekend_zeros  = uwdw.filter(((uwdw['dw'] == 6)|(uwdw['dw'] == 7)) & ((uber["Time (Local)"] >= 17)| (uber["Time (Local)"] <3)))\
.agg(sum("Zeroes")).collect()[0]['sum(Zeroes)']
total_zeros= uber.agg(sum("Zeroes").alias("total_zeros")).collect()[0]["total_zeros"]
print("The percentage of zeros that occurred on weekends is:",weekend_zeros / total_zeros * 100)


The percentage of zeros that occurred on weekends is: 29.111266620014


In [6]:
#5. What is the weighted average ratio of completed trips per driver during the two-week period? 
# Tip: “Weighted average” means your answer should account for the total trip volume in each hour 
# to determine the most accurate number in the whole period.
from pyspark.sql.functions import avg,col

weighted_avg = uber.withColumn("completed_per_driver", uber["Completed Trips"] / uber["Unique Drivers"]) \
                 .groupBy("Date", "Time (Local)") \
                 .agg(avg("completed_per_driver").alias("avg_completed_per_driver"), sum("Completed Trips").alias("total_completed_trips")) \
                 .withColumn("weighted_ratio", col("avg_completed_per_driver") * col("total_completed_trips")) \
                 .agg(sum("weighted_ratio") / sum("total_completed_trips")).collect()[0][0]

print("The weighted average ratio of completed trips per driver is:", weighted_avg)


The weighted average ratio of completed trips per driver is: 0.8276707747535554


In [7]:
#In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours 
#over the two-week period in terms of unique requests? A new shift starts every 8 hours. Assume that 
#a driver will work the same shift each day.
from pyspark.sql.functions import col, hour, countDistinct
from pyspark.sql.window import Window

# Calculate the number of unique requests for each hour of the day
hourly_unique_requests = (uber
  .groupBy("Time (Local)")).agg(countDistinct("Requests").alias("unique_requests"))

# Slide a window of 8 hours to find the busiest 8 consecutive hours
window = Window.orderBy(col("unique_requests").desc()).rowsBetween(0, 7)
busiest_8_consecutive_hours = (hourly_unique_requests
  .select("*", sum("unique_requests").over(window).alias("consecutive_sum"))
  .orderBy(col("consecutive_sum").desc())
  .limit(1)
)

# Print the result
busiest_8_consecutive_hours.show()


+------------+---------------+---------------+
|Time (Local)|unique_requests|consecutive_sum|
+------------+---------------+---------------+
|          20|             12|             80|
+------------+---------------+---------------+



7. True or False: Driver supply always increases when demand increases during the two-week period.

This statement is false. There are multiple reasons why driver supply might not always increase when demand increases. For example, some drivers might choose not to work during peak demand times, or there might be external factors that affect driver availability (such as traffic, weather conditions, or events in the city). To confirm this, we would need to analyze the data and identify instances where demand increased but driver supply did not.

In [8]:
# 8. In which 72-hour period is the ratio of Zeroes to Eyeballs the highest?
from pyspark.sql.functions import col, sum, concat_ws

uber = uber.withColumn('date_hour', concat_ws(' ', uber.Date,uber['Time (Local)']))
# Group the data by 72-hour periods and calculate the ratio of zeroes to eyeballs for each period
period_ratios = (uber.groupBy(((to_timestamp(uber['date_hour'],"dd-MMM-yy H").cast("long")/ (72*3600)).cast("int")).alias("period"))\
.agg(sum("Zeroes").alias("zeroes"), sum("Eyeballs").alias("eyeballs"))
.withColumn("ratio", col("zeroes") / col("eyeballs"))
)

# Find the period with the highest ratio
highest_ratio_period = period_ratios.orderBy(col("ratio").desc()).limit(1)

# Print the result
highest_ratio_period.show() #2-hour period is the ratio of Zeroes to Eyeballs the highest?


+------+------+--------+-------------------+
|period|zeroes|eyeballs|              ratio|
+------+------+--------+-------------------+
|  5199|   434|    1783|0.24340998317442514|
+------+------+--------+-------------------+



In [9]:
#9.If you could add 5 drivers to any single hour of every day during the two-week period, which hour 
#should you add them to? Hint: Consider both rider eyeballs and driver supply when choosing.
# Calculate requests per unique driver for each hour

import pyspark.sql.functions as F
requests_per_driver = (uber.groupBy('Time (Local)').agg(
    (F.sum('Requests') / F.countDistinct('Unique Drivers')).alias('requests_per_driver'))
)

# Show the hour with the highest ratio
requests_per_driver.orderBy(F.desc('requests_per_driver')).show(1)

+------------+-------------------+
|Time (Local)|requests_per_driver|
+------------+-------------------+
|           2|               20.0|
+------------+-------------------+
only showing top 1 row



In [10]:
#10.Looking at the data from all two weeks, which time might make the most sense to consider a true 
# “end day” instead of midnight? (i.e when are supply and demand at both their natural minimums)
# Calculate average completed trips and unique drivers for each hour
avg_trips_and_drivers = (uber.groupBy('Time (Local)').agg(
    F.mean('Completed Trips').alias('avg_completed_trips'),
    F.mean('Unique Drivers').alias('avg_unique_drivers')
))

# Show the hour with the lowest average completed trips and unique drivers
avg_trips_and_drivers.orderBy('avg_completed_trips', 'avg_unique_drivers').show(1)

+------------+-------------------+------------------+
|Time (Local)|avg_completed_trips|avg_unique_drivers|
+------------+-------------------+------------------+
|           4|0.14285714285714285|0.6428571428571429|
+------------+-------------------+------------------+
only showing top 1 row



[back to top](#0)